# VacationPy
---

## Import Libraries and Load the Weather and Coordinates Data

In [7]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
import time
import holoviews

# Import API key
from api_keys import geoapify_key

In [8]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,umm lajj,25.0213,37.2685,18.67,51,100,5.92,SA,1675747481
1,1,port alfred,-33.5906,26.8910,22.32,92,100,2.24,ZA,1675747481
2,2,albany,42.6001,-73.9662,-2.88,68,55,0.89,US,1675747462
3,3,coihaique,-45.5752,-72.0662,12.34,66,40,2.06,CL,1675747482
4,4,georgetown,5.4112,100.3354,30.96,70,20,3.60,MY,1675747339


---

### Step 1: Create a map that displays a point for every city in the city_data_df DataFrame. The size of the point should be the humidity in each city.

In [9]:
%%capture --no-display

# Configure the map plot
humid_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 700,
    frame_height = 550,
    size = "Humidity",
    scale = 0.8,
    color = "City"
)

# Display the map
humid_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)



### Step 2: Narrow down the city_data_df DataFrame to find your ideal weather condition¶

In [10]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_preferences_df = city_data_df.loc[(city_data_df['Max Temp'] < 10) &
                                            (city_data_df['Max Temp'] > -10)&
                                            (city_data_df['Humidity'] < 90) &
                                            (city_data_df['Wind Speed'] <= 2)]

# Drop any rows with null values
city_data_preferences_df = city_data_preferences_df.dropna(axis='rows')

# Display sample data
city_data_preferences_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,albany,42.6001,-73.9662,-2.88,68,55,0.89,US,1675747462
5,5,nemuro,43.3236,145.5750,-3.83,71,81,1.78,JP,1675747305
34,34,kodiak,57.7900,-152.4072,-1.27,80,75,0.00,US,1675747489
95,95,lasa,46.6166,10.7002,-1.51,86,63,0.49,IT,1675747517
96,96,kedzierzyn-kozle,50.3498,18.2261,-7.23,80,7,1.26,PL,1675747518
132,132,aksu,41.1231,80.2644,0.27,66,84,1.45,CN,1675747535
148,148,olga,43.7444,135.2892,-2.00,42,0,1.59,RU,1675747540
174,174,dalton,34.7698,-84.9702,7.79,84,20,0.00,US,1675747547
232,232,hami,42.8000,93.4500,1.76,43,100,1.00,CN,1675747565
302,302,huinan,42.6225,126.2614,-5.16,69,60,1.87,CN,1675747414


### Step 3: Create a new DataFrame called hotel_df.

In [11]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_preferences_df[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
2,albany,US,42.6001,-73.9662,68,
5,nemuro,JP,43.3236,145.5750,71,
34,kodiak,US,57.7900,-152.4072,80,
95,lasa,IT,46.6166,10.7002,86,
96,kedzierzyn-kozle,PL,50.3498,18.2261,80,
132,aksu,CN,41.1231,80.2644,66,
148,olga,RU,43.7444,135.2892,42,
174,dalton,US,34.7698,-84.9702,84,
232,hami,CN,42.8000,93.4500,43,
302,huinan,CN,42.6225,126.2614,69,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
# Set parameters to search for a hotel
categories = "accommodation.hotel"
radius = 10000
limit = 10
params = {
    "categories": categories,
    "limit": limit,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude from the DataFrame
    lat = row["Lat"] 
    lng = row["Lng"]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary

    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
#     # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
albany - nearest hotel: No hotel found
nemuro - nearest hotel: イーストハーバーホテル
kodiak - nearest hotel: Shelikof Lodge
lasa - nearest hotel: Hotel "Drei Kreuz"
kedzierzyn-kozle - nearest hotel: Court
aksu - nearest hotel: Akesu Tianyuan International Hotel
olga - nearest hotel: гостиница РосМорПорта
dalton - nearest hotel: Days Inn
hami - nearest hotel: Хами
huinan - nearest hotel: No hotel found
cordele - nearest hotel: Baymont by Wyndham Cordele
rudsar - nearest hotel: هتل بهار
roma - nearest hotel: Wingate by Wyndham Rome
komsomolskiy - nearest hotel: Hotel "Ikat House"
babushkin - nearest hotel: Трактир
hettstedt - nearest hotel: Hotel Schlackemühle
la grande - nearest hotel: The Landing
myrtle beach - nearest hotel: Atlantic View Motel
bayji - nearest hotel: No hotel found
sheridan - nearest hotel: Comfort Inn & Suites


,City,Country,Lat,Lng,Humidity,Hotel Name
2,albany,US,42.6001,-73.9662,68,No hotel found
5,nemuro,JP,43.3236,145.5750,71,イーストハーバーホテル
34,kodiak,US,57.7900,-152.4072,80,Shelikof Lodge
95,lasa,IT,46.6166,10.7002,86,"Hotel ""Drei Kreuz"""
96,kedzierzyn-kozle,PL,50.3498,18.2261,80,Court
132,aksu,CN,41.1231,80.2644,66,Akesu Tianyuan International Hotel
148,olga,RU,43.7444,135.2892,42,гостиница РосМорПорта
174,dalton,US,34.7698,-84.9702,84,Days Inn
232,hami,CN,42.8000,93.4500,43,Хами
302,huinan,CN,42.6225,126.2614,69,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 700,
    frame_height = 550,
    scale = 0.8,
    color = "City",
    hover_cols = ["Hotel Name","Country"]
)

# Display the map
hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)